## CPN,LVQ神经网络

### CPN实现函数逼近

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

class ImageSequenceGIF:
    def __init__(self, gif_dir='temp_gif_frames', gif_file='output.gif'):
        self.gif_dir = gif_dir
        self.gif_file = gif_file
        self.frames = []
        self.frame_count = 0

        if not os.path.exists(gif_dir):
            os.makedirs(gif_dir)

    def add_frame_from_plt(self, plt_figure=None):
        if plt_figure is None:
            plt_figure = plt.gcf()

        filename = os.path.join(self.gif_dir, f"frame_{self.frame_count:04d}.png")
        plt_figure.savefig(filename, bbox_inches='tight')
        self.frames.append(Image.open(filename))
        self.frame_count += 1
        plt.close(plt_figure)

    def save_gif(self, duration=100, last_frame_duration=200):
        if not self.frames:
            print("No frames to save.")
            return

        durations = [duration] * (len(self.frames) - 1) + [last_frame_duration]
        self.frames[0].save(
            self.gif_file,
            save_all=True,
            append_images=self.frames[1:],
            duration=durations,
            loop=0
        )
        print(f"GIF saved as {self.gif_file}")

    def clear_temp_frames(self):
        for f in os.listdir(self.gif_dir):
            os.remove(os.path.join(self.gif_dir, f))
        self.frames = []
        self.frame_count = 0


#### 单向CPN

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import exp, linspace, dot, amin, where, array, zeros, eye
from numpy.linalg import inv

# Hermite多项式函数
def f_hermit(x):
    return 1.1 * (1 - x + 2 * x**2) * exp(-x**2 / 2)

# 生成训练数据
TRAIN_DATA_NUM = 500
np.random.seed(0)
x_train = np.random.uniform(-4, 4, TRAIN_DATA_NUM)
y_train = f_hermit(x_train)

# 显示结果
def show_res(x, y):
    x_line = linspace(-4, 4, 250)
    plt.clf()
    plt.plot(x_line, f_hermit(x_line), '--', c='grey', linewidth=1, label='Hermit Func')
    plt.scatter(x, y, s=10, c='darkviolet', label='Train Data')
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.tight_layout()

# 最近邻竞争函数
def WTA_nearest(x, v):
    err = [x - vv for vv in v]
    dist = [dot(e, e) for e in err]
    id = where(dist == amin(dist))[0][0]
    return id

# K近邻更新函数
def K_neighbor(v, x, eta):
    for xx in x:
        id = WTA_nearest(xx, v)
        v[id] = v[id] + eta * (xx - v[id])
    return v

# CPN输出函数
def CPN_v_out(x, v):
    H = []
    for xx in x:
        h = zeros(v.shape[0])
        id = WTA_nearest(xx, v)
        h[id] = 1
        H.append(list(h))
    return array(H).T

# CPN权重更新函数
def CPN_W(h, y):
    vv = h.T.dot(h) + 0.000001 * eye(h.shape[1])
    vvv = inv(vv).dot(h.T)
    return y.reshape(1, -1).dot(vvv)

# 设置参数
NODE_NUM = 20
v_data = x_train[0:NODE_NUM]  # 初始化神经元中心
ETA_BEGIN = 0.1
ETA_END = 0.0
TRAIN_STEP = 100

# 初始化GIF生成类
gif_maker = ImageSequenceGIF(gif_dir='temp_gif_frames', gif_file='GIF/cpn_single.gif')

# 训练过程
for id, eta in enumerate(np.linspace(ETA_BEGIN, ETA_END, TRAIN_STEP)):
    v_data = K_neighbor(v_data, x_train, eta)  # 更新神经元中心

    # 扩展训练数据
    x_train_ex = x_train
    y_train_ex = f_hermit(x_train_ex)

    # 计算隐层输出和权重更新
    H = CPN_v_out(x_train_ex, v_data)
    W = CPN_W(H, y_train_ex)
    yy = W.dot(H)

    # 绘制逼近曲线
    y_train = f_hermit(x_train)
    show_res(x_train, y_train)

    x_line = np.linspace(-4, 4, 500)
    Hx = CPN_v_out(x_line, v_data)
    y_line = W.dot(Hx)

    plt.plot(x_line, y_line[0], c='red', linewidth=1, label='Net Performance')
    v_H = CPN_v_out(v_data, v_data)
    v_yy = W.dot(v_H)
    plt.scatter(v_data, v_yy[0], s=30, c='darkcyan', label='V value')

    plt.title(f'Step: {id}, Eta: {eta:.2f}')
    plt.tight_layout()
    plt.legend(loc='upper right')
    plt.draw()
    gif_maker.add_frame_from_plt(plt.gcf())  # 将当前帧添加到 GIF 中

# 保存 GIF
gif_maker.save_gif(duration=100, last_frame_duration=500)
plt.show()


GIF saved as GIF/cpn_single.gif


![](GIF/cpn_single.gif)

#### 双向CPN

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import exp, linspace, dot, amin, where, array, zeros, eye
from numpy.linalg import inv
from PIL import Image
import os

# Hermite多项式函数
def f_hermit(x):
    return 1.1 * (1 - x + 2 * x**2) * np.exp(-x**2 / 2)

# 生成训练数据
TRAIN_DATA_NUM = 500
np.random.seed(0)
x_train = np.random.uniform(-4, 4, TRAIN_DATA_NUM)
y_train = f_hermit(x_train)

# 显示结果
def show_res(x, y):
    x_line = np.linspace(-4, 4, 250)
    plt.clf()
    plt.plot(x_line, f_hermit(x_line), '--', c='grey', linewidth=1, label='Hermit Func')
    plt.scatter(x, y, s=10, c='darkviolet', label='Train Data')
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.tight_layout()

# 最近邻竞争函数
def WTA_nearest(x, v):
    err = [x - vv for vv in v]
    dist = [dot(e, e) for e in err]
    id = where(dist == amin(dist))[0][0]
    return id

# K近邻更新函数
def K_neighbor(v, x, eta):
    for xx in x:
        id = WTA_nearest(xx, v)
        v[id] = v[id] + eta * (xx - v[id])
    return v

# CPN输出函数
def CPN_v_out(x, v):
    H = []
    for xx in x:
        h = zeros(v.shape[0])
        id = WTA_nearest(xx, v)
        h[id] = 1
        H.append(list(h))
    return array(H).T

# CPN权重更新函数
def CPN_W(h, y):
    vv = h.T.dot(h) + 0.000001 * eye(h.shape[1])
    vvv = inv(vv).dot(h.T)
    return y.reshape(1, -1).dot(vvv)

# 设置参数
NODE_NUM = 20  # 设置隐层节点数量为20
v_data = x_train[0:NODE_NUM]  # 初始化神经元中心
ETA_BEGIN = 0.1
ETA_END = 0.0
TRAIN_STEP = 100

# 初始化GIF生成类
gif_maker = ImageSequenceGIF(gif_dir='temp_gif_frames', gif_file='GIF/cpn_double.gif')

# 训练过程
for id, eta in enumerate(np.linspace(ETA_BEGIN, ETA_END, TRAIN_STEP)):
    v_data = K_neighbor(v_data, x_train, eta)  # 更新神经元中心

    # 扩展训练数据
    x_train_ex = x_train
    y_train_ex = f_hermit(x_train_ex)

    # 计算隐层输出和权重更新
    H = CPN_v_out(x_train_ex, v_data)
    W = CPN_W(H, y_train_ex)
    yy = W.dot(H)

    # 绘制逼近曲线
    y_train = f_hermit(x_train)
    show_res(x_train, y_train)

    x_line = np.linspace(-4, 4, 500)
    Hx = CPN_v_out(x_line, v_data)
    y_line = W.dot(Hx)

    # 绘制网络逼近性能
    plt.plot(x_line, y_line[0], c='red', linewidth=1, label='Net Performance')
    v_H = CPN_v_out(v_data, v_data)
    v_yy = W.dot(v_H)
    
    # 显示隐层节点的值
    plt.scatter(v_data, v_yy[0], s=30, c='darkcyan', label='Hidden Node Value')
    plt.title(f'Step: {id}, Eta: {eta:.2f}')
    plt.tight_layout()
    plt.legend(loc='upper right')
    plt.draw()
    gif_maker.add_frame_from_plt(plt.gcf())  # 将当前帧添加到 GIF 中

# 保存 GIF
gif_maker.save_gif(duration=100, last_frame_duration=500)
plt.show()


GIF saved as GIF/cpn_double.gif


![](GIF/cpn_double.gif)

In [11]:
import numpy as np

# Hermite多项式函数
def f_hermit(x):
    return 1.1 * (1 - x + 2 * x**2) * np.exp(-x**2 / 2)

# 生成测试数据
TEST_DATA_NUM = 200
x_test = np.linspace(-4, 4, TEST_DATA_NUM)
y_test = f_hermit(x_test)

# 计算均方误差
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# 单向CPN和双向CPN的预测函数
def single_cpn_predict(x_train, y_train, x_test, hidden_nodes=20):
    v_data = x_train[:hidden_nodes]
    for eta in np.linspace(0.1, 0.0, 100):
        v_data = K_neighbor(v_data, x_train, eta)
    H = CPN_v_out(x_train, v_data)
    W = CPN_W(H, y_train)
    H_test = CPN_v_out(x_test, v_data)
    return W.dot(H_test)[0]

def double_cpn_predict(x_train, y_train, x_test, hidden_nodes=20):
    v_data = np.vstack((x_train[:hidden_nodes], f_hermit(x_train[:hidden_nodes]))).T
    xy_train = np.vstack((x_train, y_train)).T
    for eta in np.linspace(0.1, 0.0, 100):
        v_data = K_neighbor(v_data, xy_train, eta)
    H = CPN_v_out(x_train, v_data[:, 0])
    W = CPN_W(H, y_train)
    H_test = CPN_v_out(x_test, v_data[:, 0])
    return W.dot(H_test)[0]

# 执行单向和双向CPN的逼近，并计算精度
y_single_pred = single_cpn_predict(x_train, y_train, x_test)
y_double_pred = double_cpn_predict(x_train, y_train, x_test)

# 计算精度
mse_single = mse(y_test, y_single_pred)
mse_double = mse(y_test, y_double_pred)

print(f"Single CPN MSE: {mse_single:.5f}")
print(f"Double CPN MSE: {mse_double:.5f}")


Single CPN MSE: 0.02414
Double CPN MSE: 0.02269


#### 精度对比

* 单向CPN的MSE值：单向CPN仅使用输入值来训练隐层节点，对于逼近函数的能力有限，因此MSE值相对可能会高一些。
* 双向CPN的MSE值：双向CPN通过在隐层节点中同时考虑输入和输出信息，使得模型在训练过程中更准确地捕捉到数据的分布特性，因此MSE值可能会更低。